In [11]:
import pandas as pd
import numpy as np
import re
from itertools import groupby
from os import listdir, path
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import calendar

In [12]:
#definicões de variáveis
path_inmet = 'datasets/inmet' #caminho para os arquivos do inmet
path_output = 'datasets/agregados' #caminho para onde deseja que as planilhas por estação sejam inseridas
padronizar_dados = True #defina true se deseja que a padronização seja feita (precisa dos arquivos do inmet)

In [13]:
## Conjunto de funções para unificar e padronizar os datasets
colunas_importantes = [0, 1, 2, 6, 9, 10, 13, 14, 18]
index_metadados = ['REGIÃO', 'UF', 'ESTAÇÃO', 'CODIGO (WMO)', 'LATITUDE', 'LONGITUDE', 'ALTITUDE', 'DATA DE FUNDAÇÃO']

 #troca / por - e adiciona o 20 na frente
def formata_data(dt):
    if '/' not in dt: return dt
    d, m, a = dt.split('/')
    return f'20{a}-{m}-{d}'


#troca / por - e remove 'UTC' das strings de hora
def formata_data_hora(mi):
    data, hora = mi[0].replace('/', '-'), mi[1].replace(' UTC', '')
    if len(hora) == 4:  hora = hora[:2] + ':' + hora[2:]
    return data, hora


#faz a leitura de cada arquivo e concatena
def concat_years(code, file_list, output_dir):
    lla_data = [] #geographic data
    sensor_data = [] #sensor data
    for file in file_list:
        #leitura dos sensores
        df = pd.read_csv(file, skiprows=8, encoding='latin_1', sep=';', decimal=',', usecols=colunas_importantes, index_col=[0, 1], na_values=[-9999])
        df = df.rename_axis(['Data', 'Hora']).rename(columns={ df.columns[1]: 'RADIACAO GLOBAL (KJ/m²)'})
        sensor_data.append(df)
        
        #leitura dos dados geograficos
        md = pd.read_csv(file, encoding='latin_1', sep=';', decimal=',', skiprows=4, nrows=3, header=None, usecols=[1], na_values=['F'], names=[df.index[0][0][:4]])
        lla_data.append(md)
    
    #concatena os dados geograficos
    md = pd.concat(lla_data, axis=1, copy=False)
    md.index = ['LATITUDE', 'LONGITUDE', 'ALTITUDE']
    
    #concatena, arruma os index e escreve os dados para um arquivo
    df = pd.concat(sensor_data, copy=False).replace(-9999, np.nan)
    df.index = df.index.map(formata_data_hora)
    
    #salva todas as leituras em um arquivo por sensor
    last = '_'.join(file_list[-1].split('_')[1:5])
    path = f'{output_dir}/{last}.csv'
    md.to_csv(path, sep=';')
    df.sort_index().to_csv(path, sep=';', mode='a')

    
def unify_data(inmet_dir, output_dir):
    #salvando o nome de todas as planilhas
    arquivos = []
    for folder in listdir(inmet_dir):
        c = f'{inmet_dir}/{folder}'
        if not path.isdir(c): continue
        if path.isdir(f'{c}/{folder}'): c = f'{c}/{folder}'
        arquivos += [f'{c}/{a}' for a in listdir(c) if a.endswith('.CSV')]
    
    #para cada codigo, concatena os anos e salva os metadados
    metadatas = []
    search_groups = lambda s: re.search('_([A-Z][0-9]{3})_', s).group(1)
    for k, grupo in groupby(sorted(arquivos, key=search_groups), search_groups):
        concat_years(k, sorted(grupo), output_dir)
        print(f'{k} OK')

In [14]:
 #retorna maximos e mínimos para cada coluna em todos os arquivos
def checking_bounds(path):
    tabelas = {
        'min': [], #minimo de cada coluna
        'max': [], #maximo de cada coluna
    }
      
    for file in listdir(path):
        data = pd.read_csv(path+file, sep=';', index_col = [0, 1], skiprows=4)
        tabelas['min'].append(data.min().rename(file[:-4]))
        tabelas['max'].append(data.max().rename(file[:-4]))
    
    for k, v in tabelas.items():
        df = pd.DataFrame(v)
        df.columns = [f'{k.upper()} - {c}' for c in df.columns]
        tabelas[k] = df
        
    colunas = [v.columns for k, v in tabelas.items()]
    colunas = [j for i in zip(*colunas) for j in i]
    return pd.concat(tabelas.values(), axis=1).reindex(columns=colunas).sort_index()

###dados.apply(lambda x: x.groupby(x.notna().cumsum()).cumcount().max()).rename(local)

In [15]:
#retorna os dias marcados com True se estavam off ou False caso contrário
def days_off(file):
    data = pd.read_csv(file, sep=';', index_col = [0, 1], skiprows=4) \
                        .isna().apply(lambda row: all(row), axis = 1) \
                        .groupby(level=0).apply(lambda group: sum(group) == 24)
    data.name = file[:-4].split('/')[-1]
    return data


# retorna porcentagem de dias que a estação ficou off no ano
def percentage_off_per_year(path):
    stations = pd.concat([days_off(path+file) for file in listdir(path)], axis=1)
    return stations.groupby(lambda x: x.split('-')[0] ).apply(lambda x: x.sum(min_count=1)/ len(x) ).T.sort_index()

In [16]:
if padronizar_dados:
    unify_data(path_inmet, path_output)

In [17]:
bounds = checking_bounds('datasets/agregados/')
bounds.to_csv('datasets/extremos.csv', sep=';')
bounds

""


In [19]:
df = percentage_off_per_year('datasets/agregados/')
df.index = df.index.str.split('_', expand=True).set_names(['REGIÃO', 'ESTADO', 'CODIGO', 'NOME'])
df = df.reorder_levels(['REGIÃO', 'ESTADO', 'NOME', 'CODIGO'])
df.to_csv(f'datasets/%_dias_off.csv', sep=';')
df

ValueError: No objects to concatenate

In [53]:
path = 'datasets/agregados/'
todos = []
for file in listdir(path):
    data = pd.read_csv(path+file, sep=';', index_col = [0], nrows=3).iloc[:, -1]
    loc = file[:-4].split('_')
    data['REGIÃO'] = loc[0]
    data['ESTADO'] = loc[1]
    data['NOME'] = loc[3]
    data['CODIGO'] = loc[2]
    #data = data.rename(loc[2])
    todos.append(data)
coords = pd.concat(todos, axis=1).T.set_index(['REGIÃO', 'ESTADO', 'NOME', 'CODIGO'])
coords

LATITUDE  LONGITUDE ALTITUDE
REGIÃO ESTADO NOME                 CODIGO                               
NE     CE     BARBALHA             A315    -7.300833 -39.271111   409.41
N      PA     TOME ACU             A213      -2.5925 -48.360556    42.95
CO     MS     BRASILANDIA          S705   -21.298333 -52.068889    345.0
N      AC     MARECHAL THAUMATURGO A137        -8.95 -72.786667   220.98
NE     CE     ACARAU               A360    -3.121111 -40.087222    67.15
...                                              ...        ...      ...
       PI     PAULISTANA           A330    -8.132288 -41.142945    376.0
SE     MG     AIMORES              A534   -19.532778 -41.090833   287.74
              DIAMANTINA           A537   -18.231052 -43.648269  1359.25
S      SC     FLORIANOPOLIS        A806    -27.60253 -48.620096     4.87
NE     BA     BRUMADO              A433   -14.181944 -41.672222   473.37

[613 rows x 3 columns]

In [54]:
df2 = pd.concat([coords, df], axis=1)
df2.to_csv('')

LATITUDE  LONGITUDE ALTITUDE  \
REGIÃO ESTADO NOME                 CODIGO                                  
NE     CE     BARBALHA             A315    -7.300833 -39.271111   409.41   
N      PA     TOME ACU             A213      -2.5925 -48.360556    42.95   
CO     MS     BRASILANDIA          S705   -21.298333 -52.068889    345.0   
N      AC     MARECHAL THAUMATURGO A137        -8.95 -72.786667   220.98   
NE     CE     ACARAU               A360    -3.121111 -40.087222    67.15   
...                                              ...        ...      ...   
       PI     PAULISTANA           A330    -8.132288 -41.142945    376.0   
SE     MG     AIMORES              A534   -19.532778 -41.090833   287.74   
              DIAMANTINA           A537   -18.231052 -43.648269  1359.25   
S      SC     FLORIANOPOLIS        A806    -27.60253 -48.620096     4.87   
NE     BA     BRUMADO              A433   -14.181944 -41.672222   473.37   

                                           2000  2001  2002      2003  \
REGIÃO ESTADO NOME                 CODIGO                               
NE     CE     BARBALHA             A315     NaN   NaN   NaN       NaN   
N      PA     TOME ACU             A213     NaN   NaN   NaN       NaN   
CO     MS     BRASILANDIA          S705     NaN   NaN   NaN       NaN   
N      AC     MARECHAL THAUMATURGO A137     NaN   NaN   NaN       NaN   
NE     CE     ACARAU               A360     NaN   NaN   NaN       NaN   
...                                         ...   ...   ...       ...   
       PI     PAULISTANA           A330     NaN   NaN   NaN       NaN   
SE     MG     AIMORES              A534     NaN   NaN   NaN       NaN   
              DIAMANTINA           A537     NaN   NaN   NaN       NaN   
S      SC     FLORIANOPOLIS        A806     NaN   NaN   NaN  0.167123   
NE     BA     BRUMADO              A433     NaN   NaN   NaN       NaN   

                                               2004      2005      2006  ...  \
REGIÃO ESTADO NOME                 CODIGO                                ...   
NE     CE     BARBALHA             A315         NaN       NaN       NaN  ...   
N      PA     TOME ACU             A213         NaN       NaN       NaN  ...   
CO     MS     BRASILANDIA          S705         NaN       NaN       NaN  ...   
N      AC     MARECHAL THAUMATURGO A137         NaN       NaN       NaN  ...   
NE     CE     ACARAU               A360         NaN       NaN       NaN  ...   
...                                             ...       ...       ...  ...   
       PI     PAULISTANA           A330         NaN       NaN       NaN  ...   
SE     MG     AIMORES              A534         NaN       NaN       NaN  ...   
              DIAMANTINA           A537         NaN       NaN       NaN  ...   
S      SC     FLORIANOPOLIS        A806    0.491803  0.380822  0.052055  ...   
NE     BA     BRUMADO              A433         NaN       NaN       NaN  ...   

                                               2012      2013      2014  \
REGIÃO ESTADO NOME                 CODIGO                                 
NE     CE     BARBALHA             A315    0.000000  0.000000  0.060274   
N      PA     TOME ACU             A213    0.000000  0.000000  0.115068   
CO     MS     BRASILANDIA          S705         NaN       NaN       NaN   
N      AC     MARECHAL THAUMATURGO A137    0.120219  0.139726  0.887671   
NE     CE     ACARAU               A360    0.000000  0.000000  0.106849   
...                                             ...       ...       ...   
       PI     PAULISTANA           A330    0.000000  0.000000  0.013699   
SE     MG     AIMORES              A534    0.472678  0.000000  0.000000   
              DIAMANTINA           A537    0.000000  0.000000  0.000000   
S      SC     FLORIANOPOLIS        A806    0.000000  0.000000  0.000000   
NE     BA     BRUMADO              A433    0.000000  0.000000  0.000000   

                                               2015      2016      2017  \
REGIÃO ESTADO

In [ ]:
funcs = {
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': lambda x: x.sum(min_count=1),
    'RADIACAO GLOBAL (KJ/m²)': 'mean',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'max' ,
    'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'min' ,
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'max' ,
    'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'min' ,
    'VENTO, VELOCIDADE HORARIA (m/s)': 'mean'
}

i = 0
for file in listdir(path_output):
    print(i)
    i += 1
    path_in = f'{path_output}/{file}'
    path_out = f'datasets/diarios/{file}'
    
    pd.read_csv(path_in, sep=';', nrows=3, index_col=[0]) \
            .to_csv(path_out, sep=';')
    
    pd.read_csv( path_in, sep=';', index_col=[0, 1], skiprows=4) \
            .groupby(level=0).apply(lambda group: group.agg(funcs)) \
            .to_csv(path_out, sep=';', mode='a')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27